In [1]:
from ultralytics import YOLO
import json
import os
import shutil
import random
import cv2
import matplotlib.pyplot as plt

In [2]:
json_path = "annotations.json"
images_dir = "images"
labels_dir = "labels"
data_yaml = "data.yaml"
train_dir = "dataset/train"
val_dir = "dataset/val"
os.makedirs(train_dir + "/images", exist_ok=True)
os.makedirs(train_dir + "/labels", exist_ok=True)
os.makedirs(val_dir + "/images", exist_ok=True)
os.makedirs(val_dir + "/labels", exist_ok=True)

In [3]:
# Load JSON annotation file
with open(json_path, "r") as f:
    data = json.load(f)

file_names = data["train"]["file_names"]
rois_list = data["train"]["rois_list"]
occupancy_list = data["train"]["occupancy_list"]

# Split dataset (80% train, 20% val)
data_pairs = list(zip(file_names, rois_list, occupancy_list))
random.shuffle(data_pairs)
split_idx = int(0.8 * len(data_pairs))
train_data = data_pairs[:split_idx]
val_data = data_pairs[split_idx:]

In [4]:
# Function to process and save labels
def save_labels(file_name, rois, occupancy, split):
    label_path = f"dataset/{split}/labels/" + file_name.replace(".JPG", ".txt")
    img_path = os.path.join(images_dir, file_name)
    shutil.copy(img_path, f"dataset/{split}/images/")
    
    with open(label_path, "w") as lf:
        for obj, occupied in zip(rois, occupancy):
            x_values = [p[0] for p in obj]
            y_values = [p[1] for p in obj]
            x_center = sum(x_values) / len(x_values)
            y_center = sum(y_values) / len(y_values)
            width = max(x_values) - min(x_values)
            height = max(y_values) - min(y_values)
            class_id = 0 if occupied else 1
            lf.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process all data
for file_name, rois, occupancy in train_data:
    save_labels(file_name, rois, occupancy, "train")
for file_name, rois, occupancy in val_data:
    save_labels(file_name, rois, occupancy, "val")

In [5]:
# Write data.yaml
with open(data_yaml, "w") as f:
    f.write("train: dataset/train\n")
    f.write("val: dataset/val\n")
    f.write("nc: 2\n")
    f.write("names: ['car', 'empty']\n")

In [6]:
# Load model
model = YOLO("yolo11n.pt")

# Train model
results = model.train(
    data=data_yaml, 
    epochs=5, 
    imgsz=640,
    batch=16,
    optimizer='Adam'
)

# Change path in C:\Users\[user]\AppData\Roaming\Ultralytics\settings.json to where the "ML Model" folder is stored on your computer

Ultralytics 8.3.81  Python-3.13.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8700K 3.70GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train26, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

train: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\train\labels... 227 images, 0 backgrounds, 0 corrupt: 100%|██████████| 227/227 [00:00<00:00, 281.60it/s]


train: New cache created: C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\train\labels.cache


val: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\val\labels... 112 images, 0 backgrounds, 0 corrupt: 100%|██████████| 112/112 [00:00<00:00, 461.73it/s]

val: New cache created: C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\val\labels.cache


...

WARNING  NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  75%|███████▌  | 3/4 [00:23<00:07,  7.77s/it]

WARNING  NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:28<00:00,  7.09s/it]


                   all        112       3387      0.503      0.146      0.158     0.0703
                   car        112       1760       0.34      0.257      0.227     0.0969
                 empty        104       1627      0.666     0.0356     0.0889     0.0438
Speed: 1.7ms preprocess, 64.8ms inference, 0.0ms loss, 127.4ms postprocess per image
Results saved to runs\detect\train26


In [7]:
# Visualize sample images
def visualize_samples(split, num=5):
    img_dir = f"dataset/{split}/images"
    label_dir = f"dataset/{split}/labels"

    sample_images = random.sample(os.listdir(img_dir), num)
    
    for img_file in sample_images:
        img_path = os.path.join(img_dir, img_file)
        label_path = os.path.join(label_dir, img_file.replace(".JPG", ".txt"))

        if not os.path.exists(img_path) or not os.path.exists(label_path):
            print(f"Skipping {img_file}, missing label or image file!")
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Could not load image {img_path}")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        h_img, w_img, _ = img.shape
        print(f"Processing {img_file} | Shape: {img.shape}")

        with open(label_path, "r") as lf:
            labels = lf.readlines()
            print(f"Label contents for {img_file}:\n{labels}")

            for label in labels:
                class_id, x, y, w, h = map(float, label.split())

                # Convert YOLO format (normalized) to pixel coordinates
                x1 = int((x - w / 2) * w_img)
                y1 = int((y - h / 2) * h_img)
                x2 = int((x + w / 2) * w_img)
                y2 = int((y + h / 2) * h_img)

                # Ensure box stays within image bounds
                x1 = max(0, min(w_img, x1))
                y1 = max(0, min(h_img, y1))
                x2 = max(0, min(w_img, x2))
                y2 = max(0, min(h_img, y2))

                # Ignore tiny bounding boxes
                min_box_size = 10  # Minimum pixels
                if (x2 - x1) < min_box_size or (y2 - y1) < min_box_size:
                    print(f"Skipping small box: {x1, y1, x2, y2}")
                    continue

                color = (255, 0, 0) if class_id == 0 else (0, 255, 0)
                label_name = "Car" if class_id == 0 else "Empty"

                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, label_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        plt.figure(figsize=(10, 10))  # Enlarge plot size
        plt.imshow(img)
        plt.axis("off")
        plt.show(block=True)  # Ensure the image is displayed

# Run and debug visualization
visualize_samples("train")

Processing GOPR6785.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6785.JPG:
['1 0.5597005 0.8970408333333333 0.2990000000000001 0.2470566666666667\n', '1 0.4495755 0.7807766666666667 0.27150000000000013 0.31599999999999995\n', '0 0.3844505 0.6562766666666666 0.23950000000000005 0.2559999999999999\n', '1 0.3019505 0.46638783333333333 0.17516675 0.1657776666666666\n', '1 0.33657549999999997 0.55011 0.20599999999999996 0.21266666666666667\n', '0 0.2764921875 0.4035545833333333 0.1531665 0.13333333333333341\n', '1 0.2586171875 0.35572116666666664 0.12983324999999996 0.10511100000000001\n', '1 0.24595050000000002 0.32144341666666665 0.11833325 0.08955566666666664\n', '1 0.23486718750000002 0.2907766666666667 0.10716675000000003 0.08133333333333331\n', '1 0.1544088125 0.8337211666666666 0.18133325 0.2577776666666667\n', '1 0.129388 0.7062768333333334 0.1510835 0.2182223333333333\n', '1 0.11765875 0.6035823333333333 0.12975 0.19066666666666654\n', '1 0.11209625 0.5235823333333334 0.1175

<Figure size 1000x1000 with 1 Axes>

Processing GOPR6578.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6578.JPG:
['0 0.3822455 0.30256758333333333 0.06733325000000007 0.055999999999999994\n', '0 0.447578875 0.30645649999999997 0.05399999999999999 0.05511100000000002\n', '0 0.5094955624999999 0.31156758333333334 0.05066675000000004 0.05066666666666664\n', '0 0.5757871875 0.31778991666666667 0.04766674999999998 0.053332999999999964\n', '0 0.6264954375 0.3257899166666667 0.05233325000000011 0.05111133333333334\n', '0 0.6781621875 0.33412316666666664 0.06133325000000012 0.05066666666666664\n', '0 0.7248385625 0.3434895 0.06575000000000009 0.06\n', '0 0.25112525 0.6635545833333333 0.14621225000000004 0.2414139999999999\n', '0 0.370064625 0.6822414166666666 0.11287874999999997 0.2585856666666666\n', '0 0.5191176875 0.6645646666666667 0.11590924999999996 0.2707069999999999\n', '0 0.6499888125 0.6496655833333334 0.13484849999999993 0.2494953333333334\n', '0 0.7545335 0.6475336666666667 0.12449999999999994 0.2120000000000000

<Figure size 1000x1000 with 1 Axes>

Processing GOPR6788.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6788.JPG:
['0 0.36858674999999996 0.8828805833333333 0.257 0.2652703333333334\n', '0 0.26883675 0.7927296666666666 0.22549999999999998 0.31799999999999995\n', '1 0.20933675 0.696563 0.18750000000000003 0.2553333333333332\n', '0 0.16458674999999998 0.6126463333333334 0.16599999999999998 0.22933333333333328\n', '0 0.1334825 0.544313 0.133 0.16866666666666674\n', '1 0.11473249999999999 0.5011463333333334 0.11725 0.1403333333333333\n', '1 0.10085749999999999 0.466563 0.10474999999999998 0.12699999999999995\n', '0 0.089795 0.43697966666666666 0.09299999999999999 0.10733333333333334\n', '1 0.0818575 0.413563 0.086 0.09133333333333332\n', '1 0.07492 0.39439633333333335 0.07874999999999999 0.08400000000000002\n', '0 0.85468875 0.5870175833333333 0.10233325000000004 0.204889\n', '0 0.7720221875 0.53318425 0.11766674999999993 0.18977766666666668\n', '0 0.6903555 0.48140641666666667 0.128 0.172\n', '1 0.6110221875 0.435628666

<Figure size 1000x1000 with 1 Axes>

Processing GOPR6603.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6603.JPG:
['1 0.13003299999999998 0.5165613333333333 0.09399999999999999 0.21866666666666668\n', '1 0.205783 0.45289466666666667 0.11950000000000002 0.20666666666666672\n', '0 0.286783 0.39089466666666667 0.13650000000000004 0.18133333333333335\n', '0 0.365408 0.33700583333333334 0.14816675 0.15444433333333335\n', '0 0.43649125 0.29156150000000003 0.15100000000000008 0.13155566666666668\n', '0 0.499866375 0.25500591666666667 0.14949999999999997 0.11333366666666664\n', '0 0.554408 0.22645033333333334 0.1395 0.096889\n', '0 0.59986625 0.20508933333333335 0.13124999999999998 0.08500000000000002\n', '1 0.1724109375 0.3187836666666667 0.07866675000000001 0.12266666666666665\n', '1 0.112765 0.3645893333333333 0.0625 0.1353333333333333\n', '0 0.2348275 0.2758393333333333 0.09150000000000003 0.10799999999999998\n', '0 0.2957025 0.238006 0.09975000000000003 0.0933333333333333\n', '1 0.3549525 0.20558933333333335 0.109499999

<Figure size 1000x1000 with 1 Axes>

Processing GOPR6561.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6561.JPG:
['1 0.4701149375 0.77630975 0.2712500000000001 0.3533333333333333\n', '1 0.3558954375 0.8778301666666667 0.2700625 0.27987933333333326\n', '0 0.229258125 0.9396045 0.18854175000000004 0.1683743333333334\n', '0 0.5617314375 0.65335325 0.25287224999999997 0.3159763333333334\n', '0 0.6372576875 0.54593575 0.21801499999999996 0.2605136666666667\n', '0 0.694306875 0.4610140833333333 0.18929000000000007 0.21029733333333334\n', '1 0.0612844375 0.4888981666666667 0.08374999999999999 0.20861133333333326\n', '0 0.1265005625 0.41910308333333335 0.10729175000000002 0.19306233333333345\n', '0 0.19594587500000002 0.35247625 0.11927075000000004 0.16444466666666668\n', '1 0.2616860625 0.29611008333333333 0.12746349999999998 0.1368056666666667\n', '0 0.326270375 0.24742758333333334 0.14166674999999995 0.11803533333333335\n', '0 0.0156766875 0.5422797500000001 0.04065225 0.19874999999999998\n', '0 0.38803125 0.205961333333

<Figure size 1000x1000 with 1 Axes>